# Biodiversity Protection Indicator Geoprocessing 

## Load modules

In [5]:
from cartoframes.auth import set_default_credentials
from cartoframes import read_carto, to_carto
import geopandas as gpd
import pandas as pd
import os
import logging 
from shapely import geometry
import requests
import re
from bs4 import BeautifulSoup
import glob
from zipfile import ZipFile
import shutil

## Set up

Set up logging

In [6]:
# get top-level logger object
logger = logging.getLogger()
for handler in logger.handlers: logger.removeHandler(handler)
# manually set level 
logger.setLevel(logging.INFO)
# print to console
console = logging.StreamHandler()
logger.addHandler(console)
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

Read in processed data on Ecologically and Biologically Significant Areas (EBSAs)

In [7]:
ebsa = '/home/rthoms/Github/resource-watch/wri-projects/ocean-watch/processing-scripts/biodiversity-protection/EBSA/dissolved_ebsa.shp'
ebsa_gdf = gpd.read_file(ebsa)

In [8]:
print(ebsa_gdf.head())

   dissolve  NAME Workshop  EBSA_ID  GLOBAL_ID  valid  \
0         1  EBSA     EBSA        0          0      1   

                                            geometry  
0  MULTIPOLYGON (((-115.10000 -47.20000, -115.100...  


Read in processed Marine Protected Areas (MPA) data

In [9]:
mpa = '/home/rthoms/Github/resource-watch/wri-projects/ocean-watch/processing-scripts/biodiversity-protection/MPAs/valid_mpa.shp'
mpa_gdf = gpd.read_file(mpa)
print(mpa_gdf.columns)

Index(['WDPAID', 'WDPA_PID', 'PA_DEF', 'NAME', 'ORIG_NAME', 'DESIG',
       'DESIG_ENG', 'DESIG_TYPE', 'IUCN_CAT', 'INT_CRIT', 'MARINE',
       'REP_M_AREA', 'GIS_M_AREA', 'REP_AREA', 'GIS_AREA', 'NO_TAKE',
       'NO_TK_AREA', 'STATUS', 'STATUS_YR', 'GOV_TYPE', 'OWN_TYPE',
       'MANG_AUTH', 'MANG_PLAN', 'VERIF', 'METADATAID', 'SUB_LOC',
       'PARENT_ISO', 'ISO3', 'SUPP_INFO', 'CONS_OBJ', 'valid', 'geometry'],
      dtype='object')


Read in EEZ data from Carto

In [10]:
# authenticate carto account  
CARTO_USER = os.getenv('CARTO_WRI_RW_USER')
CARTO_KEY = os.getenv('CARTO_WRI_RW_KEY')
set_default_credentials(username=CARTO_USER, base_url="https://{user}.carto.com/".format(user=CARTO_USER),api_key=CARTO_KEY)

logger.info('Pulling EEZ boundaries from Carto table')
#read in the GADM country boundaries used in the OW geostore
EEZ_gdf = read_carto("com_011_3_maritime_boundaries_exclusive_economic_zones")


Pulling EEZ boundaries from Carto table


## Geoprocessing

### 1. Subset MPA dataset

Join the mpa and ebsa datasets, retaining mpa geometry, to determine which MPA polygons intersect with EBSAs

In [11]:
mpa_join = gpd.sjoin(ebsa_gdf, mpa_gdf, how = 'right')

In [12]:
print(len(mpa_join.index))

17281


Keep relevant MPAs, ie only those that cover EBSAs

In [13]:
mpa_subset = mpa_join.loc[~mpa_join['EBSA_ID'].isnull()]

In [14]:
print(len(mpa_subset.index))
print(mpa_subset.columns)

3364
Index(['index_left', 'dissolve', 'NAME_x', 'Workshop', 'EBSA_ID', 'GLOBAL_ID',
       'valid_x', 'WDPAID', 'WDPA_PID', 'PA_DEF', 'NAME_y', 'ORIG_NAME',
       'DESIG', 'DESIG_ENG', 'DESIG_TYPE', 'IUCN_CAT', 'INT_CRIT', 'MARINE',
       'REP_M_AREA', 'GIS_M_AREA', 'REP_AREA', 'GIS_AREA', 'NO_TAKE',
       'NO_TK_AREA', 'STATUS', 'STATUS_YR', 'GOV_TYPE', 'OWN_TYPE',
       'MANG_AUTH', 'MANG_PLAN', 'VERIF', 'METADATAID', 'SUB_LOC',
       'PARENT_ISO', 'ISO3', 'SUPP_INFO', 'CONS_OBJ', 'valid_y', 'geometry'],
      dtype='object')


Dissolve the MPA polygons by the valid field. Since this field is '1' for all MPA polygons, dissolving by the field will flatten the entire dataset.

In [15]:
flat_mpa_subset = mpa_subset.dissolve('valid_y')
print('Length of dissolved MPA gdf = '+ str(len(flat_mpa_subset.index)))

Length of dissolved MPA gdf = 1


In [16]:
flat_mpa_subset.to_file('flat_mpa_subset.shp',driver='ESRI Shapefile')

In [17]:
flat_mpa_subset = '/home/rthoms/Github/resource-watch/wri-projects/ocean-watch/processing-scripts/biodiversity-protection/Join/flat_mpa_subset.shp'
flat_mpa_subset = gpd.read_file(flat_mpa_subset)

In [18]:
print(flat_mpa_subset.head())

   valid_y  index_left  dissolve NAME_x Workshop  EBSA_ID  GLOBAL_ID  valid_x  \
0        1         0.0       1.0   EBSA     EBSA      0.0        0.0      1.0   

   WDPAID WDPA_PID  ...                              MANG_AUTH  \
0    97.0       97  ...  Corporación Nacional Forestal (CONAF)   

                                           MANG_PLAN           VERIF  \
0  http://bdrnap.mma.gob.cl/recursos/SINIA/Plande...  State Verified   

  METADATAID SUB_LOC PARENT_ISO ISO3       SUPP_INFO        CONS_OBJ  \
0       1808   CL-VS        CHL  CHL  Not Applicable  Not Applicable   

                                            geometry  
0  MULTIPOLYGON (((21.425 -34.377, 21.425 -34.377...  

[1 rows x 39 columns]


In [19]:
prot_ebsa = gpd.overlay(flat_mpa_subset, ebsa_gdf, how = 'intersection')
print(prot_ebsa.head())

   valid_y  index_left  dissolve_1 NAME_x Workshop_1  EBSA_ID_1  GLOBAL_ID_1  \
0        1         0.0         1.0   EBSA       EBSA        0.0          0.0   

   valid_x  WDPAID WDPA_PID  ... ISO3       SUPP_INFO        CONS_OBJ  \
0      1.0    97.0       97  ...  CHL  Not Applicable  Not Applicable   

  dissolve_2  NAME Workshop_2 EBSA_ID_2 GLOBAL_ID_2 valid  \
0          1  EBSA       EBSA         0           0     1   

                                            geometry  
0  MULTIPOLYGON (((177.89038 28.99602, 177.89131 ...  

[1 rows x 45 columns]


In [21]:
prot_ebsa.to_file('prot_ebsa.shp',driver='ESRI Shapefile')

Normalized/laundered field name: 'GLOBAL_ID_1' to 'GLOBAL_ID_'
Normalized/laundered field name: 'GLOBAL_ID_2' to 'GLOBAL_I_1'


In [22]:
# join
ebsa_x_country= gpd.sjoin(prot_ebsa,EEZ_gdf,how = 'right')

ValueError: 'index_left' and 'index_right' cannot be names in the frames being joined

In [ ]:
print(ebsa_x_country.head())
print(ebsa_x_country.crs)
ebsa_x_country=ebsa_x_country.to_crs({'proj':'cea'})
print(ebsa_x_country.crs)

   dissolve_1 NAME_x Workshop_1  EBSA_ID_1  GLOBAL_ID_1  valid_x  WDPAID  \
0         1.0   EBSA       EBSA        0.0          0.0      1.0    97.0   
0         1.0   EBSA       EBSA        0.0          0.0      1.0    97.0   
0         1.0   EBSA       EBSA        0.0          0.0      1.0    97.0   
0         1.0   EBSA       EBSA        0.0          0.0      1.0    97.0   
0         1.0   EBSA       EBSA        0.0          0.0      1.0    97.0   

  WDPA_PID PA_DEF                       NAME_y  ... sovereign2 mrgid_ter3  \
0       97      1  Archipiélago Juan Fernández  ...       None          0   
0       97      1  Archipiélago Juan Fernández  ...       None          0   
0       97      1  Archipiélago Juan Fernández  ...       None          0   
0       97      1  Archipiélago Juan Fernández  ...    Comores          0   
0       97      1  Archipiélago Juan Fernández  ...     France          0   

  mrgid_sov3 territory3 iso_ter3 sovereign3        x_1        y_1  mrgid_eez  \


In [ ]:
ebsa_x_country['area'] = ebsa_x_country['geometry'].area/ 10**6

In [ ]:
print(ebsa_x_country.columns)

Index(['dissolve_1', 'NAME_x', 'Workshop_1', 'EBSA_ID_1', 'GLOBAL_ID_1',
       'valid_x', 'WDPAID', 'WDPA_PID', 'PA_DEF', 'NAME_y', 'ORIG_NAME',
       'DESIG', 'DESIG_ENG', 'DESIG_TYPE', 'IUCN_CAT', 'INT_CRIT', 'MARINE',
       'REP_M_AREA', 'GIS_M_AREA', 'REP_AREA', 'GIS_AREA', 'NO_TAKE',
       'NO_TK_AREA', 'STATUS', 'STATUS_YR', 'GOV_TYPE', 'OWN_TYPE',
       'MANG_AUTH', 'MANG_PLAN', 'VERIF', 'METADATAID', 'SUB_LOC',
       'PARENT_ISO', 'ISO3', 'SUPP_INFO', 'CONS_OBJ', 'dissolve_2', 'NAME',
       'Workshop_2', 'EBSA_ID_2', 'GLOBAL_ID_2', 'valid', 'geometry',
       'index_right', 'cartodb_id', 'mrgid', 'geoname', 'mrgid_ter1',
       'pol_type', 'mrgid_sov1', 'territory1', 'iso_ter1', 'sovereign1',
       'mrgid_ter2', 'mrgid_sov2', 'territory2', 'iso_ter2', 'sovereign2',
       'mrgid_ter3', 'mrgid_sov3', 'territory3', 'iso_ter3', 'sovereign3',
       'x_1', 'y_1', 'mrgid_eez', 'area_km2', 'area'],
      dtype='object')


In [ ]:
ebsa_x_country_df = ebsa_x_country.groupby(['iso_ter1']).agg({'territory1':'first', 'area_km2': 'first', 'area':'sum'}).reset_index()

In [ ]:
print(ebsa_x_country_df.head())

  iso_ter1            territory1  area_km2          area
0      ALB               Albania     12150  2.990944e+06
1      ARE  United Arab Emirates     51782  2.990944e+06
2      ATF        Crozet Islands    573927  8.972833e+06
3      BGR              Bulgaria     34685  2.990944e+06
4      CAN                Canada   5708318  2.990944e+06


In [ ]:
ebsa_x_country_df.to_csv('ebsa_x_country.csv')